# Lesson 12: Model Inference and Function Calling

## Introduction (5 minutes)

Welcome to our lesson on Model Inference and Function Calling. In this 60-minute session, we'll explore practical aspects of using Large Language Models (LLMs), including loading and using local models, calling remote APIs, and working with the JAIS model.

## Lesson Objectives

By the end of this lesson, you will be able to:
1. Load and use local LLM models using PyTorch and Hugging Face
2. Estimate model size and manage GPU resources
3. Use the OpenAI API to access remote LLM services
4. Implement inference using the JAIS model

## 1. Using PyTorch/HuggingFace to Load and Use Local LLM Models (25 minutes)

### 1.1 Loading a Pre-trained Model (10 minutes)

Let's start by loading a pre-trained model using the Transformers library:

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

def load_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    return tokenizer, model

# Load GPT-2 as an example
model_name = "gpt2"
tokenizer, model = load_model(model_name)

print(f"Model loaded: {model_name}")
print(f"Model size: {model.num_parameters()} parameters")

### 1.2 Estimating Model Size and GPU Memory (5 minutes)

It's crucial to understand the memory requirements of your model:

In [ ]:
def estimate_model_size(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()

    size_all_mb = (param_size + buffer_size) / 1024**2
    return size_all_mb

model_size_mb = estimate_model_size(model)
print(f"Estimated model size: {model_size_mb:.2f} MB")

# Check available GPU memory
if torch.cuda.is_available():
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**2
    print(f"Total GPU memory: {gpu_memory:.2f} MB")
else:
    print("GPU not available")

### 1.3 Configuring GPU Usage (5 minutes)

If you have multiple GPUs, you can specify which one to use:

In [ ]:
import os

def set_gpu(gpu_id):
    if torch.cuda.is_available():
        os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
        device = torch.device(f"cuda:{gpu_id}")
        print(f"Using GPU: {gpu_id}")
    else:
        device = torch.device("cpu")
        print("GPU not available, using CPU")
    return device

# Example: Use GPU 0
device = set_gpu(0)
model.to(device)

### 1.4 Model Inference (5 minutes)

Now, let's perform inference with our loaded model:

In [ ]:
def generate_text(model, tokenizer, prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_length=max_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

prompt = "In the future, artificial intelligence will"
generated_text = generate_text(model, tokenizer, prompt)
print(f"Generated text: {generated_text}")

## 2. Using OpenAI API to Call LLM Remote Service (15 minutes)

OpenAI's API provides access to powerful language models like GPT-3:

In [ ]:
import openai

# Set your API key
openai.api_key = "your-api-key-here"

def generate_text_openai(prompt, model="text-davinci-002", max_tokens=50):
    response = openai.Completion.create(
        engine=model,
        prompt=prompt,
        max_tokens=max_tokens
    )
    return response.choices[0].text.strip()

# Example usage
prompt = "Translate the following English text to French: 'Hello, how are you?'"
generated_text = generate_text_openai(prompt)
print(f"OpenAI API response: {generated_text}")

### 2.1 Error Handling and Rate Limiting (5 minutes)

When working with remote APIs, it's important to handle errors and respect rate limits:

In [ ]:
import time

def generate_text_openai_with_retry(prompt, model="text-davinci-002", max_tokens=50, max_retries=3):
    for attempt in range(max_retries):
        try:
            return generate_text_openai(prompt, model, max_tokens)
        except openai.error.RateLimitError:
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)  # Exponential backoff
            else:
                raise
        except openai.error.OpenAIError as e:
            print(f"An error occurred: {e}")
            return None

# Example usage with retry
generated_text = generate_text_openai_with_retry(prompt)
if generated_text:
    print(f"OpenAI API response (with retry): {generated_text}")

## 3. Demo: Using the JAIS Model (15 minutes)

The JAIS model is a powerful Arabic language model. While we don't have direct access to it, we can demonstrate how you might use it if it were available through a similar interface as other Hugging Face models:

In [ ]:
def load_jais_model():
    # This is a placeholder function. In reality, you would need the actual model files and possibly special tokenizer.
    model_name = "jais-model"  # This would be the actual model name or path
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    return tokenizer, model

def generate_text_jais(model, tokenizer, prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=max_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage (this is conceptual and won't actually run without the JAIS model)
jais_tokenizer, jais_model = load_jais_model()
arabic_prompt = "ترجم النص التالي إلى اللغة الإنجليزية: 'مرحبا، كيف حالك؟'"
generated_text = generate_text_jais(jais_model, jais_tokenizer, arabic_prompt)
print(f"JAIS model response: {generated_text}")

Note: The above code is conceptual. To actually use the JAIS model, you would need access to the model files and possibly a special tokenizer designed for Arabic text.

## Conclusion and Q&A (5 minutes)

We've covered how to load and use local LLM models, estimate their size and manage GPU resources, use the OpenAI API for remote inference, and conceptually how to work with the JAIS model. These skills form the foundation for implementing LLMs in various applications.

Are there any questions about model inference or function calling?

## Additional Resources

1. Hugging Face Transformers documentation: https://huggingface.co/transformers/
2. PyTorch CUDA semantics: https://pytorch.org/docs/stable/notes/cuda.html
3. OpenAI API documentation: https://beta.openai.com/docs/
4. JAIS model information (if available, please provide the official source)

In our next lesson, we'll explore advanced techniques in prompt engineering to optimize our interactions with these powerful language models.